In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
from pandas.api.types import is_numeric_dtype

In [2]:
df = pd.read_csv('../data/test_education.csv')

In [3]:
df = df.drop(['Case'],axis=1)

In [4]:
df

,Education,Skills,Experience,Productivity
0,higher,high,-,high
1,?,high,low,high
2,secondary,-,high,high
3,higher,?,high,high
4,elementary,high,low,low
5,secondary,-,high,low
6,-,low,high,low
7,elementary,?,-,low


### Attribute-Concept Values used to fill up Incomplete Data !!

#### Defining Goal Set !!

In [5]:
df_headers = list(df)
concept = df_headers[-1]
concept

'Productivity'

In [6]:
#all unique concepts
concept_list = df[concept].unique()
concept_list

array(['high', 'low'], dtype=object)

In [7]:
#calculating cases by concepts and making sets
U = [] #universal list containing all cases
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        U.append(index)
        if row[concept] == item:
            temp_list.append(index)
    goal_list.append(temp_list)
    temp_list = []

In [8]:
goal_list

[[0, 1, 2, 3], [4, 5, 6, 7]]

#### Building Case List !!

In [9]:
# Case list is built here as it's not needed att-concept or lost value handling to do so

In [10]:
attributes = list(df)
attributes

['Education', 'Skills', 'Experience', 'Productivity']

In [11]:
case_list = []
#Loop through all the attributes except last 2 - Concept and sort_col
for item in attributes[:-1]:
    print(item)
    
    #check for non numeric columns
    if not is_numeric_dtype(df[item]):
        temp = df[item].unique()
        for i in temp:
            if i == '?' or i == '-':
                continue
            else:
                case = item + "," + i
                case_list.append(case)

Education
Skills
Experience


In [12]:
case_list

['Education,higher',
 'Education,secondary',
 'Education,elementary',
 'Skills,high',
 'Skills,low',
 'Experience,low',
 'Experience,high']

In [13]:
#Don't copy dataframes using assignment operator e.g. df=df2 
#as it'll modify the other dataframe if we attempt to edit one
#hence use copy() function

df2 = df.copy()

#### Start of handling Missing attribute !!

In [14]:
for col in df:
    for i, row_value in df[col].iteritems():
        
        #This is for attribute-concept value interpretation
        #For Lost value interpretation, we don't add that case to any block at all
        if row_value == '-':
            
            final_values = []
            print(col,i)
            concept_tmp = df[concept][i]
            print(concept_tmp)
            
            for item_list in goal_list:
                if i in item_list:
                    final_values = item_list
                    
            print (final_values)
            
            #Taking out all unique cases corresponding to the matched concept
            col_values = []
            for item in final_values:
                col_values.append(df[col][item])
        
            
            uniqueList = []
            for letter in col_values:
                if letter not in uniqueList:
                    uniqueList.append(letter)
            
            if '-' in uniqueList:
                uniqueList.remove('-')
                
            if '?' in uniqueList:
                uniqueList.remove('?')
            print(uniqueList)
            
#             print(df.loc[i,col])
            df2.at[i, col] = uniqueList
            
    print("\n")

Education 6
low
[4, 5, 6, 7]
['elementary', 'secondary']


Skills 2
high
[0, 1, 2, 3]
['high']
Skills 5
low
[4, 5, 6, 7]
['high', 'low']


Experience 0
high
[0, 1, 2, 3]
['low', 'high']
Experience 7
low
[4, 5, 6, 7]
['low', 'high']






In [15]:
df2

,Education,Skills,Experience,Productivity
0,higher,high,"[low, high]",high
1,?,high,low,high
2,secondary,[high],high,high
3,higher,?,high,high
4,elementary,high,low,low
5,secondary,"[high, low]",high,low
6,"[elementary, secondary]",low,high,low
7,elementary,?,"[low, high]",low


In [16]:
df = df2.copy()

In [17]:
df_headers = list(df)
concept = df_headers[-1]
concept_list = df[concept].unique()

In [18]:
#calculating cases by concepts and making sets
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        if row[concept] == item:
            temp_list.append(index+1)
    goal_list.append(temp_list)
    temp_list = []

In [19]:
goal_list

[[1, 2, 3, 4], [5, 6, 7, 8]]

In [20]:
df

,Education,Skills,Experience,Productivity
0,higher,high,"[low, high]",high
1,?,high,low,high
2,secondary,[high],high,high
3,higher,?,high,high
4,elementary,high,low,low
5,secondary,"[high, low]",high,low
6,"[elementary, secondary]",low,high,low
7,elementary,?,"[low, high]",low


In [21]:
case_list

['Education,higher',
 'Education,secondary',
 'Education,elementary',
 'Skills,high',
 'Skills,low',
 'Experience,low',
 'Experience,high']

In [22]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if "." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if row[a] >= int(start) and row[a] <= int(end):
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if case == b:
                        temp_list.append(index+1)
                        
            if row[a] == b:
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []

[1, 4]
[3, 6, 7]
[5, 7, 8]
[1, 2, 3, 5, 6]
[6, 7]
[1, 2, 5, 8]
[1, 3, 4, 6, 7, 8]


In [23]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       Education,higher    [1, 4]
1       Education,secondary [3, 6, 7]
2       Education,elementary[5, 7, 8]
3       Skills,high         [1, 2, 3, 5, 6]
4       Skills,low          [6, 7]
5       Experience,low      [1, 2, 5, 8]
6       Experience,high     [1, 3, 4, 6, 7, 8]


In [24]:
#Might need to handle cases when att-concept values could not be populated and default should be Null

### Start of Lower/Upper & Probabilistic Approximations !!

#### Bulding Characteristic Sets !!

In [25]:
df

,Education,Skills,Experience,Productivity
0,higher,high,"[low, high]",high
1,?,high,low,high
2,secondary,[high],high,high
3,higher,?,high,high
4,elementary,high,low,low
5,secondary,"[high, low]",high,low
6,"[elementary, secondary]",low,high,low
7,elementary,?,"[low, high]",low


In [26]:
U = set(U)
U

{0, 1, 2, 3, 4, 5, 6, 7}

In [27]:
# Creating dictionary combining case_list and att_val list
block = dict(zip(case_list, att_val_list))
block

{'Education,higher': [1, 4],
 'Education,secondary': [3, 6, 7],
 'Education,elementary': [5, 7, 8],
 'Skills,high': [1, 2, 3, 5, 6],
 'Skills,low': [6, 7],
 'Experience,low': [1, 2, 5, 8],
 'Experience,high': [1, 3, 4, 6, 7, 8]}

In [68]:
dic = {}
for index, row in df.iterrows():
    tmp_set = set()
    union_set = set()
    char_list = []
    char_list_2 = []
    for cols in attributes[:-1]:
        #If the value for corresponding attribute is a list then create all of the att-value pairs
        if type(df.loc[index,cols]) == list:
            for item in df.loc[index,cols]:
                block_key = cols + "," + item
                char_list.append(block_key) #char_list has all att-val cases
        else:
            block_key = cols + "," + df.loc[index,cols]
            char_list_2.append(block_key) #char_list_2 has all single cases
       
   
    #Compute instersection for this current row for Characteristics set
    
    #Compute union of att-concept value case
    for item in char_list:
        union_set = union_set.union(set(block[item]))
        
    for item in char_list_2:
        if item in block:
            if tmp_set == set():
                #Copy over the current set elements to B
                for i in range(len(block[item])):
                    tmp_set.add(block[item][i])
                    
            tmp_set = tmp_set.intersection(set(block[item]))
        print(tmp_set)
        #Problem starts here 
        else:
            if tmp_set == set():
                tmp_set = U
            tmp_set = tmp_set.intersection(U)
    
    if union_set == set():
        tmp_set = tmp_set
    else:
        tmp_set = tmp_set.intersection(union_set)
        
         
    key = ('K_%d' % (index+1))
    dic[key] = tmp_set

here Education,?
here Skills,?
here Skills,?


In [57]:
dic

{'K_1': {1},
 'K_2': {1, 2, 5},
 'K_3': {3, 6},
 'K_4': {1, 4},
 'K_5': {5},
 'K_6': {3, 6, 7},
 'K_7': {6, 7},
 'K_8': {5, 7}}